# BagiBagi: Recommendation Systems for Consumer

Based on the given PRD and group discussion with BagiBagi Team, we all agreed to use Recommendation Systems using TensorFlow with a goals to map this following inputs:

1. Customer ID
2. Product Category
3. Quantity of past purchase
4. Customer Age
5. Age
6. Gender (optional)

To get the following outputs:



# Outline
- [ 1 - Data Description ](#1)
- [ 2 - Data Exploration ](#2)
- [ 3 - Data Pre-processing ](#3)
  - [ 3.1 ](#3.1)
  - [ 3.2 ](#3.2)
- [ 4 - Content-based filtering with a neural network ](#4)
  - [ 4.1 Split training data ](#3.1)
  - [ 4.2 Set the models](#3.2)
- [ 5 - Predictions](#5)
  - [ 5.1 - Predictions for a new user](#5.1)
  - [ 5.2 - Predictions for an existing user.](#5.2)
  - [ 5.3 - Finding Similar Items](#5.3)
